In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

wine = pd.read_csv('https://bit.ly/wine_csv_data')
data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()
train_input, test_input, train_target, test_target = train_test_split(
    data, target, test_size = 0.2, random_state=42
)

print(data)

[[ 9.4   1.9   3.51]
 [ 9.8   2.6   3.2 ]
 [ 9.8   2.3   3.26]
 ...
 [ 9.4   1.2   2.99]
 [12.8   1.1   3.34]
 [11.8   0.8   3.26]]


In [3]:
# 앙상블 훈련 중 하나인 랜덤 포레스트 모델을 사용
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_jobs=-1, random_state = 42)
scores = cross_validate(rf, train_input, train_target,
                       return_train_score=True, n_jobs = -1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9973541965122431 0.8905151032797809


In [ ]:

rf.fit(train_input, train_target)

# [0.23167441 0.50039841 0.26792718]
# 여전히 sugar가 0.5로 중요도가 가장 높다.
# 하지만 alcohol이 0.23, pH가 0.26으로 비중이 많이 높아졌다.
print(rf.feature_importances_)

[0.23167441 0.50039841 0.26792718]


In [ ]:
# oob 점수를 사용하면 교차 검증을 대신할 수 있다.
rf = RandomForestClassifier(oob_score=True, n_jobs=-1, random_state = 42)
rf.fit(train_input, train_target)
print(rf.oob_score_)

0.8934000384837406


In [ ]:
# Extra Tree는 랜덤 포레스트 모델와 다르게 
# 중복되지 않는 자료(부트 스트랩)를 사용
from sklearn.ensemble import ExtraTreesClassifier
et = ExtraTreesClassifier(n_jobs = -1, random_state=42)
scores = cross_validate(et, train_input, train_target,
                        return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9974503966084433 0.8887848893166506


In [ ]:
et.fit(train_input, train_target)
print(et.feature_importances_)

[0.20183568 0.52242907 0.27573525]


In [ ]:
# 그레이디언트 부스팅
# 깊이가 얕은 결정 트리를 사용하여 이진 트리의 오차를 보완하는 방식
# 경사 하강법을 사용하여 트리를 앙상블에 추가함
# 분류에서는 로지스틱 손실 함수를 사용하고 회귀에서는 평균 제곱 오차 함수를 사용
# 일반적으로 높은 성능을 기대할 수 있다.

from sklearn.ensemble import GradientBoostingClassifier
gb = GradientBoostingClassifier(random_state = 42)
scores = cross_validate(gb, train_input, train_target,
                        return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.8881086892152563 0.8720430147331015


In [ ]:
# 결정 트리 개수를 500개로 증가하여 다시 실행
# learning_rate의 default 값이 0.1
# n_jobs 매개변수는 없다.
gb = GradientBoostingClassifier(n_estimators=500, learning_rate=0.2,
                                random_state = 42)
scores = cross_validate(gb, train_input, train_target,
                        return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9464595437171814 0.8780082549788999


In [ ]:
gb.fit(train_input, train_target)
print(gb.feature_importances_)

[0.15872278 0.68010884 0.16116839]


In [ ]:
# 히스토그램 기반 그레이디언트 부스팅(Histogram-based Gradient Boosting)
# 정형 데이터를 다루는데 인기있는 알고리즘
# 그래이디언트 부스팅의 속도와 성능을 더욱 개선함.
# 입력 특성을 256개의 구간을 나누고, 하나를 떼어놓고, 누락된 값을 위하여 사용함
# - 입력에 누락된 늑성이 있더라도 전처리할 필요가 없다는 것이 특징
#from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier

hgb = HistGradientBoostingClassifier(random_state=42)
scores = cross_validate(hgb, train_input, train_target,
                        return_train_score = True)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9321723946453317 0.8801241948619236


In [ ]:
# 히스토그램 기반 그레이디언트 부스팅의 특성 중요도를 계산하기 위해 permutation_importance 함수를 사용한다.
# 이 함수는 특성을 하나씩 랜덤하게 섞어서 모델의 성능이 변화하는지를 관찰하며, 어떤 특성이 중요한지 계산한다.

from sklearn.inspection import permutation_importance

hgb = HistGradientBoostingClassifier(random_state=42)

hgb.fit(train_input, train_target)

# n_repeats : 랜덤하게 섞을 횟수를 지정, 기본값은 5
# 질문 : 아니 근데, fit를 이미 한 상태에서 모델의 성능을 관찰해봐야 의미가 있는가?
# 그렇다면 fit가 포함되어 있는 함수인가?
result = permutation_importance(hgb, train_input, train_target,
                                n_repeats=10, random_state=42, n_jobs=-1)

print(result.importances_mean)

[0.08876275 0.23438522 0.08027708]


In [ ]:
# 테스트에서 특성 중요도를 계산해본다.
result = permutation_importance(hgb, test_input, test_target,
                                n_repeats=10, random_state=42, n_jobs=-1)

print(result.importances_mean)

[0.05969231 0.20238462 0.049     ]


In [ ]:
# 최종적으로 테스트 세트에서의 성능은?
print(hgb.score(test_input, test_target))

print(scores['train_score'])
print(scores['test_score'])

0.8723076923076923
[0.93192206 0.93216262 0.92857143 0.93265993 0.93554594]
[0.87115385 0.88365385 0.90279115 0.86621752 0.87680462]


In [ ]:
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9321723946453317 0.8801241948619236


In [ ]:
print(result)

{'importances_mean': array([0.05969231, 0.20238462, 0.049     ]), 'importances_std': array([0.004     , 0.007938  , 0.00453846]), 'importances': array([[0.06230769, 0.05769231, 0.05538462, 0.05538462, 0.06076923,
        0.06076923, 0.06846154, 0.06230769, 0.05461538, 0.05923077],
       [0.20076923, 0.2       , 0.21153846, 0.20076923, 0.20307692,
        0.18923077, 0.19615385, 0.19461538, 0.21384615, 0.21384615],
       [0.05692308, 0.04692308, 0.05076923, 0.04769231, 0.04692308,
        0.05      , 0.04384615, 0.04692308, 0.04307692, 0.05692308]])}
[0. 1.]


In [6]:
##################
# 사이킷럿 외에 제공하는 히스토그램 부스팅 알고리즘을 구현한 라이브러리
# 1. XGBoost : cross_validate() 함수 사용 가능
##################

from xgboost import XGBClassifier
xgb = XGBClassifier(tree_method='hist', random_state=42)
scores = cross_validate(xgb, train_input, train_target, return_train_score=True)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.8824322471423747 0.8726214185237284


In [7]:
##################
# 사이킷럿 외에 제공하는 히스토그램 부스팅 알고리즘을 구현한 라이브러리
# 2. Lightgbm : cross_validate() 함수 사용 가능
##################
from lightgbm import LGBMClassifier
lgb = LGBMClassifier(random_state=42)
scores = cross_validate(lgb, train_input, train_target, return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9338079582727165 0.8789710890649293
